In [ ]:
from service_capacity_modeling.interface import CapacityDesires
from service_capacity_modeling.interface import FixedInterval, Interval
from service_capacity_modeling.interface import QueryPattern, DataShape 

import pprint
def mprint(x):
    pprint.pprint(x.model_dump(), sort_dicts=False)
    
def do_summarize(cluster, regret):
    cost = cluster.candidate_clusters.total_annual_cost
    zonal = cluster.candidate_clusters.zonal
    count = len(zonal) * zonal[0].count
    instance = zonal[0].instance.name
    
    return f"{count:>3} {instance:>10} costing {cost} -> {regret}", f"{count:>3} {instance:>10}"

def summarize(cluster, regret):
    print(do_summarize(cluster, regret)[0])

MiB = (1024 * 1024)
GiB = MiB * 1024

# Provisioning a Kafka Cluster

Right out of the box we now how to capacity plan Kafka

In [ ]:
throughput = 100 * MiB

kafka_desires = CapacityDesires(
    # Tier 1 service, prod like
    service_tier=1,
    query_pattern=QueryPattern(
        # 2 consumers
        estimated_read_per_second=Interval(low=1, mid=2, high=2, confidence=0.98),
        # 1 producer
        estimated_write_per_second=Interval(low=1, mid=1, high=1, confidence=0.98),
        # Write throughput of 1 GiB/s
        estimated_mean_write_size_bytes=Interval(
            low=throughput, mid=throughput, high=throughput * 2, confidence=0.98
        ),
    ),
)

lower_throughput = CapacityDesires(
    # Tier 1 service, prod like
    service_tier=1,
    query_pattern=QueryPattern(
        # 2 consumers
        estimated_read_per_second=Interval(low=1, mid=2, high=2, confidence=0.98),
        # 1 producer
        estimated_write_per_second=Interval(low=1, mid=1, high=1, confidence=0.98),
        # Write throughput of 1 GiB/s
        estimated_mean_write_size_bytes=Interval(
            low=1 * MiB, mid=1 * MiB, high=2 * MiB, confidence=0.98
        ),
    ),
)

# kafka_desires = lower_throughput

In [ ]:
from service_capacity_modeling.capacity_planner import planner
from service_capacity_modeling.models.org import netflix

# Load up the Netflix capacity models
planner.register_group(netflix.models)
model_name = "org.netflix.kafka"


cap_plan = planner.plan(
    model_name=model_name,
    region="us-east-1",
    desires=kafka_desires,
    simulations=1024,
    explain=True,
    extra_model_arguments={
        "cluster_type": "strong",
        "retention": "PT2H",
        #"copies_per_region": 2
    },
)


# An uncertain Capacity plan has a few elements

* `requirement`: The range of requirements
* `least_regret`: The clusters which minimized the models regret function over many simulations.
* `mean`: The clusters that would be provisioned for the average (midpoint) of all intervals.
* `percentiles`: The clusters that would be provisioned assuming various percentiles of the inputs (can use this to generate a "worse case" estimate)

In [ ]:
requirement = cap_plan.requirements

lr_opt1, lr_opt1_cost = (
    cap_plan.least_regret[0].candidate_clusters.zonal[0],
    cap_plan.least_regret[0].candidate_clusters.total_annual_cost
)
if len(cap_plan.least_regret) > 1:
    lr_opt2, lr_opt2_cost = (
        cap_plan.least_regret[1].candidate_clusters.zonal[0],
        cap_plan.least_regret[1].candidate_clusters.total_annual_cost
    )
else:
    lr_opt2, lr_opt2_cost = lr_opt1, lr_opt1_cost
mean_opt, mean_cost = (
    cap_plan.mean[0].candidate_clusters.zonal[0],
    cap_plan.mean[0].candidate_clusters.total_annual_cost
)

In [ ]:
mprint(requirement)

In [ ]:
print("Cluster:",); mprint(lr_opt1)
print("\nRequirement:",); mprint(cap_plan.least_regret[0].requirements.zonal[0])
print(f"\nCost: {lr_opt1_cost}")

In [ ]:
print("Cluster:",); mprint(lr_opt2)
print("\nRequirement:",); mprint(cap_plan.least_regret[1].requirements.zonal[0])
print(f"Cost: {lr_opt2_cost}")

# Visualize the Simulation
We can visualize what is happening via the explain param

In [ ]:
worlds = cap_plan.explanation.regret_clusters_by_model[model_name]
defaults = cap_plan.explanation.desires_by_model[model_name]
    
min_regret = (float('inf'), None, None)
max_regret = (0, None, None)

i = 0
choices = {}
print("\nWorlds:")
print("-"*20)

for cluster, desire, regret in worlds:
    i += 1
    summary, choice = do_summarize(cluster, regret)
    print(f"{i:>3}: ", summary)
    if choice in choices:
        choices[choice] += 1
    else:
        choices[choice] = 1
    
    if regret < min_regret[0]:
        min_regret = (regret, desire, cluster)
    if regret > max_regret[0]:
        max_regret = (regret, desire, cluster)
        
print("\nLeast Regret Choice:")
print("-"*20)
summarize(min_regret[2], min_regret[0])

print("\nAll Choices")
print("-"*11)
pprint.pprint(choices)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from typing import Optional
from service_capacity_modeling.stats import dist_for_interval

qps_max, space_max = 4, 10_000 

def plot_desires(desires: CapacityDesires, concrete_desire: Optional[CapacityDesires]):
    # Writes
    wps = desires.query_pattern.estimated_write_per_second
    wps_dist = dist_for_interval(wps)
    
    w_size = desires.query_pattern.estimated_mean_write_size_bytes
    w_size_dist = dist_for_interval(w_size)
    
    w_lat = desires.query_pattern.estimated_mean_write_latency_ms
    w_lat_dist = dist_for_interval(w_lat)

    # Reads
    rps = desires.query_pattern.estimated_read_per_second
    rps_dist = dist_for_interval(rps)

    r_size = desires.query_pattern.estimated_mean_read_size_bytes
    r_size_dist = dist_for_interval(r_size)
    
    r_lat = desires.query_pattern.estimated_mean_read_latency_ms
    r_lat_dist = dist_for_interval(r_lat)
    
    # Space
    space = desires.data_shape.estimated_state_size_gib
    space_dist = dist_for_interval(space)
    
    # Compression
    compression = desires.data_shape.estimated_compression_ratio
    compression_dist = dist_for_interval(compression)
    
    
    if concrete_desire is not None:
        world_wps = concrete_desire.query_pattern.estimated_write_per_second
        world_w_size = concrete_desire.query_pattern.estimated_mean_write_size_bytes
        world_rps = concrete_desire.query_pattern.estimated_read_per_second
        world_r_size = concrete_desire.query_pattern.estimated_mean_read_size_bytes
        world_space = concrete_desire.data_shape.estimated_state_size_gib
        world_compression = concrete_desire.data_shape.estimated_compression_ratio
        world_write_lat = concrete_desire.query_pattern.estimated_mean_write_latency_ms
        world_r_lat = concrete_desire.query_pattern.estimated_mean_read_latency_ms

    fig, axs = plt.subplots(4, 2, figsize=(24, 18))

    qps_x = np.linspace(500, qps_max, 10000)
    size_x = np.linspace(16, 2048, 10000)
    space_x = np.linspace(10, space_max, 10000)
    compression_x = np.linspace(1, 10, 1000)
    lat_x = np.linspace(0.2, 10, 1000)

    user_color, model_color = "#61d83610", "#ff644e10"

    # Left hand side = RPS, WPS, Size, Compression
    # Right hand size = RS, WS, RC, WC

    axs[0][0].plot(qps_x, rps_dist.pdf(qps_x), '-', lw=2, label='Reads per Second')
    axs[0][0].axvline(x=rps.mid, color='k', linestyle='--', label="E[Reads per Second]")
    axs[0][0].set_xlabel("Reads (1/s)")
    axs[0][0].set_facecolor(user_color)
    
    axs[1][0].plot(qps_x, wps_dist.pdf(qps_x), '-', lw=2, label='Writes per Second')
    axs[1][0].axvline(x=wps.mid, color='k', linestyle='--', label="E[Writes per Second]")
    axs[1][0].set_xlabel("Writes (1/s)")
    axs[1][0].set_facecolor(user_color)
    
    axs[2][0].plot(space_x, space_dist.pdf(space_x), '-', lw=2, label='Data Size (GiB)')
    axs[2][0].axvline(x=space.mid, color='k', linestyle='--', label="E[Data Size]")
    axs[2][0].set_xlabel("Data Size (GiB)")
    axs[2][0].set_facecolor(user_color)
    
    axs[3][0].plot(compression_x, compression_dist.pdf(compression_x), '-', lw=2, label='Compression Ratio (1:X)')
    axs[3][0].axvline(x=compression.mid, color='k', linestyle='--', label="E[Compression Ratio]")
    axs[3][0].set_xlabel("Compression Ratio (1:X)")
    axs[3][0].set_facecolor(model_color)
    
    axs[0][1].plot(size_x, r_size_dist.pdf(size_x), '-', lw=2, label='Read Size (bytes)')
    axs[0][1].axvline(x=r_size.mid, color='k', linestyle='--', label="E[Read Size]")
    axs[0][1].set_xlabel("Read Size (bytes)")
    axs[0][1].set_facecolor(model_color)
    
    axs[1][1].plot(size_x, w_size_dist.pdf(size_x), '-', lw=2, label='Write Size (bytes)')
    axs[1][1].axvline(x=w_size.mid, color='k', linestyle='--', label="E[Write Size]")
    axs[1][1].set_xlabel("Write Size (bytes)")
    axs[1][1].set_facecolor(model_color)
    
    axs[2][1].plot(lat_x, r_lat_dist.pdf(lat_x), '-', lw=2, label='Read On-CPU Latency (ms)')
    axs[2][1].axvline(x=r_lat.mid, color='k', linestyle='--', label="E[Read On-CPU Latency]")
    axs[2][1].set_xlabel("Read On-CPU Latency (ms)")
    axs[2][1].set_facecolor(model_color)
    
    axs[3][1].plot(lat_x, w_lat_dist.pdf(lat_x), '-', lw=2, label='Write On-CPU Latency (ms)')
    axs[3][1].axvline(x=w_lat.mid, color='k', linestyle='--', label="E[Write On-CPU Latency]")
    axs[3][1].set_xlabel("Write On-CPU Latency (ms)")
    axs[3][1].set_facecolor(model_color)
    

    if concrete_desire is not None:
        axs[0][0].axvline(x=world_rps.mid, color='r', linestyle='-', label="Sampled Reads per Second")
        axs[1][0].axvline(x=world_wps.mid, color='r', linestyle='-', label="Sampled Writes per Second")
        axs[2][0].axvline(x=world_space.mid, color='r', linestyle='-', label="Sampled Data Size (GiB)")
        axs[3][0].axvline(x=world_compression.mid, color='r', linestyle='-', label="Sampled Compression Ratio (1:X)")
        
        axs[0][1].axvline(x=world_r_size.mid, color='r', linestyle='-', label="Sampled Read Size (bytes)")
        axs[1][1].axvline(x=world_w_size.mid, color='r', linestyle='-', label="Sampled Write Size (bytes)")
        axs[2][1].axvline(x=world_r_lat.mid, color='r', linestyle='-', label="Sampled Read On-CPU Latency (ms)")
        axs[3][1].axvline(x=world_write_lat.mid, color='r', linestyle='-', label="Sampled Write On-CPU Latency (ms)")


    for i in range (len(axs)):
        axs[i][0].legend()
        axs[i][1].legend()

    plt.show()

In [ ]:
plot_desires(defaults, None)

In [ ]:
world = worlds[0]
plot_desires(defaults, world[1])
summarize(world[0], world[2])
mprint(world[0].candidate_clusters.zonal[0])
mprint(world[0].requirements)

In [ ]:
world = worlds[-1]
plot_desires(defaults, world[1])
summarize(world[0], world[2])
mprint(world[0].candidate_clusters.zonal[0])
mprint(world[0].requirements)

In [ ]:
world = worlds[941]
plot_desires(defaults, world[1])
summarize(world[0], world[2])
mprint(world[0].candidate_clusters.zonal[0])

# Understanding Intervals
Our users don't know exactly how much traffic they will generate, but they have some idea of the order of magnitude

In [ ]:
left_skew  = Interval(minimum_value=0, low=100 , mid=1000, high=9900, maximum_value=10000, confidence=0.98)
right_skew = Interval(minimum_value=0, low=100 , mid=9000, high=9900, maximum_value=10000, confidence=0.98)
center     = Interval(minimum_value=0, low=100 , mid=5000, high=9900, maximum_value=10000, confidence=0.98)
shift      = Interval(minimum_value=0, low=2000, mid=3000, high=4000, maximum_value=10000, confidence=0.98)

left_beta   = dist_for_interval(left_skew)
right_beta  = dist_for_interval(right_skew)
center_beta = dist_for_interval(center)
shift_beta  = dist_for_interval(shift)

In [ ]:
import matplotlib
from cycler import cycler

plt.rcParams.update({'font.size': 12})
fig, ax = plt.subplots(1, 1, figsize=(20,10))

x = np.linspace(0, 10000, 10000)
ax.plot(x, left_beta.pdf(x), color="tab:blue",lw=2, label='Left Skew [P₁=100, μ=1000, P₉₉=9900]')
ax.plot([1000, 1000], [0, left_beta.pdf(1000)], color="tab:blue", marker='o')

ax.plot(x, center_beta.pdf(x), lw=2, color="tab:green",  label='Centered    [P₁=100, μ=5000, p₉₉=9900]')
ax.plot([5000, 5000], [0, center_beta.pdf(5000)], color="tab:green", marker='o')

ax.plot(x, right_beta.pdf(x), lw=2, color="tab:orange", label='Right Skew [P₁=100, μ=9000, P₉₉=9900]')
ax.plot([9000, 9000], [0, right_beta.pdf(9000)], color="tab:orange", marker='o')

ax.plot(x, shift_beta.pdf(x), lw=2, color="tab:purple", label='Shift Left    [P₁=2000, μ=3000, P₉₉=4000]')
ax.plot([3000, 3000], [0, shift_beta.pdf(3000)], color="tab:purple", marker='o')

ax.legend(loc='best', frameon=False)
ax.set_xlim(0.0, 10000)
ax.set_title("Intervals Spanning 0-10000")
ax.set_ylabel("Probability Density")

plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(20,10))

samples = 100000
q = (0.01, 0.99)
parts = ax.violinplot(
    [left_beta.rvs(samples),right_beta.rvs(samples), center_beta.rvs(samples)],
    quantiles=[q,q,q],
    showmeans=True, showextrema=False, vert=False)
    
ax.set_xlim(0.0, 10000)
ax.set_title("Intervals Spanning 0-10000")
ax.yaxis.set_tick_params(direction='out')
ax.yaxis.set_ticks_position('left')
ax.set_yticks([1, 2, 3])
ax.set_yticklabels(["Skew Left", "Skew Right","Centered"])

plt.show()